# DEEPX Tutorial 03 - AI Porject Workflow with DEEPX NPU

이번 과정에서는 DEEPX 하드웨어에 AI 모델을 배포하는 처음부터 끝까지의 전체적인 개발 방식을 보여줍니다.

지게차 및 작업자 감지 모델을 학습시키고, DX-COM 도구를 사용하여 DXNN 형식으로 변환하고, 최종 AI 애플리케이션을 DEEPX NPU에서 실행합니다. 이 과정을 통해 DEEPX NPU 개발 파이프라인에 대한 전반적인 이해를 얻을 수 있습니다.

> This tutorial is based on dx-all-suite v2.0.0, released in September 2025.

## Hands-on Project Overview

- **Detection classes**: Forklift, Worker
- **Base AI model**: YOLOv7
- **Dataset**: 1448 images about Forklift & Worker from [kaggle](https://www.kaggle.com/datasets/hakantaskiner/personforklift-dataset/data)
- **Train**: Need NVIDIA GPU with more than 24G GRAM
- **Inference NPU**: `DX-M1`
- **AI application**: Modify and reuse the yolo demo of the DX-APP
- **Expected output**:
  ![img](assets/detection-goal.jpg)

## AI Workflow Overview

이 다이어그램은 AI 프로젝트의 일반적인 Workflow 를 설명합니다.

개발 목표를 정의하고, 데이터를 수집 및 레이블링하고, 모델을 학습시킵니다.
DX-Compiler는 DX NPU를 위한 모델을 더 빠르고 가볍게 만드는 데 도움이 됩니다(INT8).
마지막 단계는 DX-App 또는 DX-Stream을 사용하여 모델을 DEEPX NPU에 배포하는 것입니다.

각 단계는 작업자 및 지게차 감지와 같은 실제 AI 솔루션을 구축하는 데 도움이 됩니다.
![img](assets/workflow2.jpg)

## 1. AI Workflow - 사용 목적에 맞는 AI Model 선정하기

AI 프로젝트를 시작하려면 사용 사례에 맞는 AI 모델을 선택해야 합니다.

이 튜토리얼의 목표는 지게차와 작업자를 감지하는 것이고, 객체 감지에 널리 알려진 모델인 YOLOv7을 사용할 것입니다.

- Choose YOLOv7 to detect Forklift & Worker
- For more details of YOLOv7: 👉 [link](https://docs.ultralytics.com/models/yolov7/)
- How to use YOLOv7: 👉 [link](https://github.com/WongKinYiu/yolov7)

## 2. AI Workflow - Data 준비 및 Annotation

Kaggle 을 통해 지게차와 작업자의 image & labling 이 완료된 dataset 을 확보 가능
 - Reference: [kaggle link](https://www.kaggle.com/datasets/hakantaskiner/personforklift-dataset)

# 3. AI Workflow - 훈련하기

모델을 효율적으로 학습시키려면 24GB 이상의 그래픽 메모리가 있는 GPU를 사용이 권장됩니다. 교육 목적으로 Google colab 을 사용하여 GPU 자원을 할당받아 YOLOv7 모델을 훈련하는 예시가 다음 링크에 있습니다.

 - How to train YOLOv7: 👉 [Link](https://colab.research.google.com/drive/1dAdjJuhXqFM_Qcd0QqAn7_AGx7abA5aX?usp=sharing)

## 4. AI Workflow - DX-Compiler 로 변환하기

In [ ]:
# Move to "dx-tutorials/dx-all-suite/dx-compiler/dx_com"
import os
root_path = os.environ.get('ROOT_PATH')
%cd $root_path/dx-all-suite/dx-compiler/dx_com

사전 학습된 AI 모델을 DXNN 형식으로 컴파일 해 봅니다.

전체적인 Process:
1. PyTorch 프레임워크에서 사전 학습된 모드를 가져옵니다.
2. ONNX 형식으로 변환합니다.
3. ONNX를 DXNN으로 컴파일합니다. (DX-COM에 대한 자세한 내용은 사용자 가이드를 참조) 👉  [here](https://developer.deepx.ai/?files=MjUxNQ==))

![img](assets/dx-com-workflow.jpg)

The source structure of DX-COM is organized as follows:
```bash
dx_com
 ├── calibration_dataset   # Dataset used to optimize model accuracy
 ├── dx_com
 │ ├── cv2/                # Third party shared libraries (e.g., OpenCV)
 │ ├── google/             # Third party shared libraries (e.g., protobuf)
 │ ├── numpy/              # Third party shared libraries (e.g., NumPy)
 │ ├── ...                 # Other dependencies
 │ └── dx_com              # Core compiler implementation
 ├── sample
 │ ├── MobilenetV1.json    # Sample configuration file
 │ └── MobilenetV1.onnx    # Sample ONNX model
 └── Makefile              # Build script for compiling the sample model
```

In [ ]:
!tree -L 1
!tree sample

### 4.1 ONNX 로 변환된 파일을 다운로드 & 컴파일을 위한 JSON file 구성방식을 살펴보기

- DX Modelzoo 에서 YOLOv7용 JSON 파일 다운로드하기: Object Detection >> YOLOV7-2 >> JSON download 👉 [modelzoo](assets/modelzoo.html)
  
  `Note`: DX modelzoo는 DEEPX에서 검증을 거쳐 공개적으로 사용 가능한 AI 모델 목록

In [ ]:
# (Optional) Download exported ONNX file:
!wget "cs.deepx.ai/_deepx_fae_archive/dx-tutorials/yolov7-forklift-person.onnx"

# Copy the downloaded files to the currect path
#!mv ~/Downloads/yolov7-forklift-person.onnx ./
!cp $ROOT_PATH/notebooks/T03-E2E-AI-Workflow/assets/YOLOV7-2.json ./yolov7-forklift-person.json
#!mv ~/Downloads/YOLOV7-2.json ./yolov7-forklift-person.json

### 4.2 YOLOv7 json 를 사용하는 custom 모델에 맞게 수정하기

전체적인 YOLOv7 모델의 topology 를 수정한 사항이 없어서, 단순히 DX Compile 과정에 필요한 calibration dataset 경로만 내 환경에 맞게 수정하면 된다.

In [ ]:
# Check the original json file downloaded from DX model zoo
!cat yolov7-forklift-person.json

In [ ]:
import json

file_name = 'yolov7-forklift-person.json'

# 1. Read JSON file
with open(file_name, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 2. Find and replace dictionary values by key
data['default_loader']['dataset_path'] = "./calibration_dataset"

# 3. Save the revised json file
with open(file_name, 'w', encoding='utf-8') as f:
    json.dump(data, f, indent=4)

print(f"'{file_name}' update done")

### 4.3 JSON configuration 을 참조해 ONNX를 DXNN으로 컴파일 하기

In [ ]:
!./dx_com/dx_com -h

# The following compile command will take more than 10 mins depending on your system performance
#!./dx_com/dx_com -m  yolov7-forklift-person.onnx -c yolov7-forklift-person.json -o output

# Move the compiled dxnn file to the dx_app path
#!mv output/yolov7-forklift-person.dxnn $ROOT_PATH/dx-all-suite/dx-runtime/dx_app/

## 5. AI Workflow - DEEPX NPU 에서 동작시키기

In [ ]:
# Move to "dx-tutorials/dx-all-suite/dx-runtime/dx_app"
%cd $root_path/dx-all-suite/dx-runtime/dx_app

### 5.1 (Optional) 컴파일 시간이 30분 가량 소요되기에, 이 과정을 건너뛰려면 아래 cell 의 주석 처리를 해제한 후 다음 링크에서 컴파일된 DXNN 파일을 다운로드 하세요.

In [ ]:
#!wget "cs.deepx.ai/_deepx_fae_archive/dx-tutorials/yolov7-forklift-person.dxnn"

### 5.2 DX-APP의 Yolo 앱을 사용자 정의 모델에 맞게 수정하세요. (Forklift & Worker detection model)
- Change the class number from 80 to 2
- Change the class list for Forklist & Worker

In [ ]:
%%writefile dx_app_update.diff
diff --git a/demos/demo_utils/yolo_cfg.cpp b/demos/demo_utils/yolo_cfg.cpp
index 5c43289..14de65f 100644
--- a/demos/demo_utils/yolo_cfg.cpp
+++ b/demos/demo_utils/yolo_cfg.cpp
@@ -140,18 +140,18 @@ YoloParam yolox_s_512 = {
 YoloParam yolov7_640 = {
     640,
     640,
-    0.25f,
-    0.3f,
-    0.4f,
+    0.2f,
+    0.2f,
+    0.2f,
     0,
-    80,
+    2,
     "output",
     {
         createYoloLayerParam("onnx::Reshape_491", 80, 80, 3, { 12.0, 19.0, 40.0 }, { 16.0, 36.0, 28.0 }, { 0 }),
         createYoloLayerParam("onnx::Reshape_525", 40, 40, 3, { 36.0, 76.0, 72.0 }, { 75.0, 55.0, 146.0 }, { 1 }),
         createYoloLayerParam("onnx::Reshape_559", 20, 20, 3, { 142.0, 192.0, 459.0 }, { 110.0, 243.0, 401.0 }, { 2 })
     },
-    { "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat", "trafficlight", "firehydrant", "stopsign", "parkingmeter", "bench", "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sportsball", "kite", "baseballbat", "baseballglove", "skateboard", "surfboard", "tennisracket", "bottle", "wineglass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli", "carrot", "hotdog", "pizza", "donut", "cake", "chair", "couch", "pottedplant", "bed", "diningtable", "toilet", "tv", "laptop", "mouse", "remote", "keyboard", "cellphone", "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors", "teddybear", "hairdrier", "toothbrush"},
+    { "Forklift", "Worker" },
     PostProcType::YOLO_LEGACY
 };
 
@@ -302,4 +302,4 @@ YoloParam scrfd_face_640_ppu = {
     },
     {"person"},
     PostProcType::FACE
-};
\ No newline at end of file
+};

`git apply` 명령을 사용하여 수정된 코드를 적용합니다.

In [ ]:
!git checkout -- demos/demo_utils/yolo_cfg.cpp
!git apply --whitespace=fix dx_app_update.diff

### 5.3 DX-APP 다시 빌드하기

In [ ]:
!./build.sh

### 5.4 DX-APP 의 yolo app 을 custom model 과 함께 동작시키기

In [ ]:
!./bin/yolo -h

In [ ]:
!cp $ROOT_PATH/notebooks/T03-E2E-AI-Workflow/assets/forklift-worker.png ./

In [ ]:
!./bin/yolo -m yolov7-forklift-person.dxnn -i forklift-worker.png -p 4

### 5.5 결과 확인:

In [ ]:
from IPython.display import Image, display

# Show the image
display(Image(filename="result.jpg"))